# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Признаки

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

# Целевой признак

Exited — факт ухода клиента

## Подготовка данных

Загружаем необходимые библиотеки

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import numpy as np

### Чтение и изучение данных

In [ ]:
df = pd.read_csv('/datasets/Churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Датасет состоит из 13 столбцов, и 10000 строк. В столбце 'Tenure' есть пропуски. Ниже необходимо будет заполнить пропуски, потому что модели машинного обучения не работают с пропусками.

Для обучения модели столбцы RowNumber, CustomerId и Surname не нужны, поэтому удалим их, чтобы лишние данные не путали модель.

In [ ]:
df.drop(['RowNumber', 'CustomerId','Surname'], axis=1, inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Переведем все названия строк в нижний регистр, а также приведем к стандарту.

In [ ]:
df = df.rename (columns= {
    'RowNumber': 'row_number',
    'CustomerId': 'customer_id',
    'CreditScore': 'credit_score',
    'NumOfProducts': 'num_of_products',
    'HasCrCard': 'has_cr_card',
    'IsActiveMember': 'is_active_member',
    'EstimatedSalary': 'estimated_salary'})
df.columns = df.columns.str.lower()
df.columns

Index(['credit_score', 'geography', 'gender', 'age', 'tenure', 'balance',
       'num_of_products', 'has_cr_card', 'is_active_member',
       'estimated_salary', 'exited'],
      dtype='object')

Проверим датасет на дубликаты

In [ ]:
df.duplicated().sum()

0

Проверим статистические данный по количественным признакам. Возможно, мы увидим выбросы.

In [ ]:
df[['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']].describe()

,credit_score,age,tenure,balance,num_of_products,estimated_salary
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,100090.239881
std,96.653299,10.487806,2.894723,62397.405202,0.581654,57510.492818
min,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,51002.110000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,100193.915000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500
max,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000


Данные выглядят нормально. Явных выбросов мы не нашли.

### Заполнение пропусков в столбце 'tenure'.

Проверим сколько пропущенные данных у столбце.

In [ ]:
df['tenure'].isna().sum()

909

Заполним пропуски значением -1. Это будет своего рода заглушка.

In [ ]:
df['tenure'] = df['tenure'].fillna(-1)
df['tenure'].value_counts()

 1.0     952
 2.0     950
 8.0     933
 3.0     928
 5.0     927
 7.0     925
-1.0     909
 4.0     885
 9.0     882
 6.0     881
 10.0    446
 0.0     382
Name: tenure, dtype: int64

In [ ]:
df['tenure'].isna().sum()

0

Переведем даные в столбце 'tenure', в формат int, потому что данные все целочисленные.

In [ ]:
df['tenure'] = df['tenure'].astype(int)
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Изучение связи между данными

Посмотрим корреляцию между целевым признаком 'exited', и признаками датасета.

In [ ]:
correlations_df = df.corr()['exited'].sort_values()
correlations_df

is_active_member   -0.156128
num_of_products    -0.047820
credit_score       -0.027094
tenure             -0.012570
has_cr_card        -0.007138
estimated_salary    0.012097
balance             0.118533
age                 0.285323
exited              1.000000
Name: exited, dtype: float64

Корреляция между признаками слабая. 

Посмотрим корреляцию между признаками. Если мы обнаружим сильную корреляцию между признаками, то нужно будет удалить один из зависимых признаков, чтобы не путать модель.

### Подготовка признаков. 

Преобразуем категориальные признаки методом One-Hot Encoding. 

In [ ]:
df_ohe = pd.get_dummies(df, drop_first=True) #Чтобы избежать дамми-ловушки используем аргумент drop_first

Разобьем датасет на 3 части (тестовую, обучающую и валидационную).

In [ ]:
# для начала разобьем выборку на тестовую и остальную часть
df_all, df_test = train_test_split(df_ohe, test_size=0.2, random_state=12345) 

# разбиваем теперь оставшуюся выборку на тренировочную и валидационную
df_train, df_valid = train_test_split(df_all, test_size=0.25, random_state=12345)

Создаем переменные с признаками.

In [ ]:
# признаки для тренировочной выборки
features_train = df_train.drop('exited', axis=1) 
target_train = df_train['exited']

# признаки для валидационной выборки. На ней мы будем проверять качество обучения.
features_valid = df_valid.drop('exited', axis=1)
target_valid = df_valid['exited']

# признаки для тестовой выборки. На ней мы проверим окончательный результат обучения.
features_test = df_test.drop('exited', axis=1)
target_test = df_test['exited']

Масштабирование количественных признаков

In [ ]:
# создаем список со столбцами с количественными данными.
numeric = ['credit_score', 'age', 'tenure', 'balance',
           'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary']
# трансформируем данные
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None;
features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
492,-0.134048,-0.078068,-0.137323,0.076163,0.816929,-1.550255,0.968496,0.331571,0,0,0
6655,-1.010798,0.494555,-1.661443,0.136391,-0.896909,0.645055,0.968496,-0.727858,0,0,1
4287,0.639554,1.353490,-1.051795,0.358435,-0.896909,0.645055,0.968496,-0.477006,1,0,1
42,-0.990168,2.116987,-0.746971,0.651725,-0.896909,0.645055,0.968496,-0.100232,0,0,0
8178,0.567351,0.685430,0.777149,0.813110,0.816929,0.645055,0.968496,0.801922,0,0,0


### Выводы:
В результате подготовки данных проделали следующую работу:
- Удалили ненужные для обучения столбцы;
- Заполнили пропуски заглужкой;
- Проанализировали данные на выбросы и корреляцию;
- Преобразовали категориальные данные методом OHE;
- Масштабировали количественные данные;
- Разбили датасет на 3 выборки и выделили признаки для обучения и целевой признак.

## Исследование задачи

Данные подготовлены к обучению, обучим несколько моделей и изучим их качество. Т.к. мы решаем задачу на классификацию, то воспользуемся моделями логистической регрессией, решающего дерева и случайного леса.

Для начала обучим модель не обращая внимание на дисбаланс данных.

<b>Логистическая регрессия</b>

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)  #расчитываем вероятность признака
 
print("F1:", f1_score(target_valid, predicted_valid)) 
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

F1: 0.30885122410546145
AUC-ROC: 0.7707540226888713


<b>Решающее дерево</b>

In [ ]:
best_result_tree = 0
best_model = None
for depth in range(1, 15):
    model = (DecisionTreeClassifier(random_state=12345, max_depth=depth)
    .fit(features_train, target_train))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_tree:
        best_model = model
        best_result_tree = F1     
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_tree)
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Лучшая модель: DecisionTreeClassifier(max_depth=7, random_state=12345)
F1 лучшей модели: 0.5488958990536277
AUC-ROC: 0.8275652142122556


<b>Случайный лес</b>

In [ ]:
# Определяем лучшее количество деревьев.
best_model = None
best_result_forest = 0
est = 0
for est in range(1, 13):
    model = (RandomForestClassifier(random_state=12345, n_estimators=est)
    .fit(features_train, target_train))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_forest:
        best_model = model
        best_est = est
        best_result_forest = F1

# Определяем лучшую глубину деревьев в модели. 
best_model = None
best_result_tree_10 = 0
for depth in range(1, 15):
    model = (RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=depth)
    .fit(features_train, target_train))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_tree_10:
        best_model = model
        best_result_tree_10 = F1     
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_tree_10)   
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Лучшая модель: RandomForestClassifier(max_depth=13, n_estimators=9, random_state=12345)
F1 лучшей модели: 0.5553822152886114
AUC-ROC: 0.8285666145832505


### Выводы:
Если мы не учитываем дисбаланс классов целевого признака, то эффективность обученных моделей не высокая.<br>
Наилучший результат показала модель Случайный лес с 13 деревьями и глубиной 9.<br>
Поработаем над дисбалансом ниже. 

## Борьба с дисбалансом

Проверка целевого признака на дисбаланс.

In [ ]:
df['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Данные распределеные не равномерно. Клиентов, ушедших из банка почти в 4 раза меньше, чем тех, кто остался.

Для начала добавим вес классов в обучение модели.

In [ ]:
# Логистическая регрессия
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия:')
print("F1:", f1_score(target_valid, predicted_valid))
probabilities_valid = model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))
print()

Логистическая регрессия:
F1: 0.47763864042933807
AUC-ROC: 0.7729666406514507



In [ ]:
# Решающее дерево
best_result_tree = 0
best_model = None
for depth in range(2, 22, 2):
    model = (DecisionTreeClassifier(random_state=12345, max_depth=depth)
    .fit(features_train, target_train))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_tree:
        best_model = model
        best_result_tree = F1     
print('Решающее дерево:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_tree)
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Решающее дерево:
Лучшая модель: DecisionTreeClassifier(max_depth=8, random_state=12345)
F1 лучшей модели: 0.5353846153846155
AUC-ROC: 0.7997779434415427


In [ ]:
# Случайный лес
best_model = None
best_result_forest = 0
for est in range(100, 1001, 100):
    for depth in range(2, 22, 2):
        model = (RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        .fit(features_train, target_train))
        predicted_valid = model.predict(features_valid)
        F1 = f1_score(target_valid, predicted_valid)
        if F1 > best_result_forest:
            best_model = model
            best_result_forest = F1
print('Случайный лес:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_forest)   
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Случайный лес:
Лучшая модель: RandomForestClassifier(max_depth=20, n_estimators=200, random_state=12345)
F1 лучшей модели: 0.5686900958466453
AUC-ROC: 0.8476750821386733


Результаты обучения улучшились, у решающего дерева немного упали. До сих пор лидирует Случайный лес.

<b>Увеличение выборки</b></br>

In [ ]:
# Разделим обучающую выборку на отрицательные и положительные объекты.
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
# Скопируем несколько раз положительные объекты:

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

# Перемешаем данные:
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
# Обучим логистическую регрессию, используя новую выборку.
model = LogisticRegression(solver='liblinear' , random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия:')
print("F1:", f1_score(target_valid, predicted_valid))
probabilities_valid = model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Логистическая регрессия:
F1: 0.47971781305114636
AUC-ROC: 0.772995252090622


In [ ]:
# Обучим решающее дерево, используя новую выборку.
best_result_tree = 0
best_model = None
for depth in range(2, 22, 2):
    model = (DecisionTreeClassifier(random_state=12345, max_depth=depth)
    .fit(features_upsampled, target_upsampled))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_tree:
        best_model = model
        best_result_tree = F1     
print('Решающее дерево:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_tree)
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Решающее дерево:
Лучшая модель: DecisionTreeClassifier(max_depth=6, random_state=12345)
F1 лучшей модели: 0.5587044534412956
AUC-ROC: 0.8088890972931989


In [ ]:
# Обучим случайный лес, используя новую выборку.
best_model = None
best_result_forest = 0
for est in range(100, 1001, 100):
    for depth in range(2, 22, 2):
        model = (RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        .fit(features_upsampled, target_upsampled))
        predicted_valid = model.predict(features_valid)
        F1 = f1_score(target_valid, predicted_valid)
        if F1 > best_result_forest:
            best_model = model
            best_result_forest = F1
print('Случайный лес:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_forest)   
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Случайный лес:
Лучшая модель: RandomForestClassifier(max_depth=16, n_estimators=500, random_state=12345)
F1 лучшей модели: 0.6013605442176871
AUC-ROC: 0.8477633007427847


Качество модели увеличилось, по прежнему лидирует модель Случайный лес.

<b>Уменьшение выборки</b></br>

Попробуем поработать с уменьшением выборки.

In [ ]:
# Разделим обучающую выборку на отрицательные и положительные объекты:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

# Отбосим из таблицы случайные элементы, используя функцию sample:
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

# Перемешаем данные:
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.26)

In [ ]:
# Обучим логистическую регрессию, используя новую выборку.
model = LogisticRegression(solver='liblinear' , random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия:')
print("F1:", f1_score(target_valid, predicted_valid))
probabilities_valid = model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Логистическая регрессия:
F1: 0.48331830477908033
AUC-ROC: 0.7736262932767887


In [ ]:
# Обучим решающее дерево, используя новую выборку.
best_result_tree = 0
best_model = None
for depth in range(2, 22, 2):
    model = (DecisionTreeClassifier(random_state=12345, max_depth=depth)
    .fit(features_downsampled, target_downsampled))
    predicted_valid = model.predict(features_valid)
    F1 = f1_score(target_valid, predicted_valid)
    if F1 > best_result_tree:
        best_model = model
        best_result_tree = F1     
print('Решающее дерево:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_tree)
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Решающее дерево:
Лучшая модель: DecisionTreeClassifier(max_depth=6, random_state=12345)
F1 лучшей модели: 0.5504761904761906
AUC-ROC: 0.821291361411752


In [ ]:
# Обучим случайный лес, используя новую выборку.
best_model = None
best_result_forest = 0
for est in range(100, 1001, 100):
    for depth in range(2, 22, 2):
        model = (RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        .fit(features_downsampled, target_downsampled))
        predicted_valid = model.predict(features_valid)
        F1 = f1_score(target_valid, predicted_valid)
        if F1 > best_result_forest:
            best_model = model
            best_result_forest = F1
print('Случайный лес:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_forest)   
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Случайный лес:
Лучшая модель: RandomForestClassifier(max_depth=10, random_state=12345)
F1 лучшей модели: 0.574826560951437
AUC-ROC: 0.8489268326024171


По сравнению с увеличением выборки, качество обученных моделей хуже. 

Во всех наших экспериментах лучшие результаты показывала модель случайного леса. Попробуем объеденить технику увеличения выборки и вес класса на лидирующей выборке.

In [ ]:
# Обучим случайный лес, используя новую выборку.
best_model = None
best_result_forest = 0
for est in range(100, 1001, 100):
    for depth in range(2, 22, 2):
        model = (RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        .fit(features_upsampled, target_upsampled))
        predicted_valid = model.predict(features_valid)
        F1 = f1_score(target_valid, predicted_valid)
        if F1 > best_result_forest:
            best_model = model
            best_result_forest = F1
print('Случайный лес:')
print("Лучшая модель:", best_model)
print("F1 лучшей модели:", best_result_forest)   
probabilities_valid = best_model.predict_proba(features_valid)
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_valid[:, 1]))

Случайный лес:
Лучшая модель: RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=200,
                       random_state=12345)
F1 лучшей модели: 0.6039119804400979
AUC-ROC: 0.8509932143203432


### Выводы:
При работе с дисбалансом классов наиболшую эффективность показал метод увеличения выборки и добавление веса классу на моделе случайного дерева с параметрами 200/12. Далее проверим нашу модель на тестовой выборке.

## Тестирование модели

In [ ]:
# Объеденим обучающую и валидационную выборку
frames_1 = [features_train, features_valid]
features = pd.concat(frames_1)

frames_2 = [target_train, target_valid]
target = pd.concat(frames_2)

#Обучим объединенную выборку и предскажем результаты на тестовой выборке.
model = (RandomForestClassifier(random_state=12345, n_estimators=200, max_depth=12, class_weight='balanced')
    .fit(features, target))
predicted_test = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_test))
probabilities_test = model.predict_proba(features_test)
print("AUC-ROC:", roc_auc_score(target_test, probabilities_test[:, 1]))

F1: 0.6085858585858585
AUC-ROC: 0.8617522566851927
